In [ ]:
# testing to see if it works
from ultralytics import YOLO
model = YOLO('yolov8s.pt')
model.info()
results = model('bus.jpg')

from ultralytics.utils.plotting import Annotator
import matplotlib.pyplot as plt
import cv2

annotator = Annotator(cv2.cvtColor(results[0].orig_img, cv2.COLOR_BGR2RGB))
boxes = results[0].boxes
for box in boxes:
    b = box.xyxy[0]  # get box coordinates in (left, top, right, bottom) format
    c = box.cls
    annotator.box_label(b, model.names[int(c)])

plt.imshow(annotator.result())

Data pipeline stuff

In [8]:
import os
import random
import shutil

# Define paths
images_dir = "datasets/Annotated_FruitNet/images"
labels_dir = "datasets/Annotated_FruitNet/labels"

image_train = os.path.join(images_dir, "train")
image_val = os.path.join(images_dir, "val")
image_test = os.path.join(images_dir, "test")

label_train = os.path.join(labels_dir, "train")
label_val = os.path.join(labels_dir, "val")
label_test = os.path.join(labels_dir, "test")

# Create test directories if they don't exist
os.makedirs(image_test, exist_ok=True)
os.makedirs(label_test, exist_ok=True)

# Get all image files (assuming .png, modify if needed)
image_files = []
for folder in [image_train, image_val]:
    for file in os.listdir(folder):
        if file.endswith((".png", ".jpg", ".jpeg")):  # Add other extensions if needed
            image_files.append(os.path.join(folder, file))

# Shuffle all images (and their labels will follow)
random.shuffle(image_files)
total_files = len(image_files)

# Calculate splits (15% test, 15% val, 70% train)
test_count = int(0.15 * total_files)
val_count = int(0.15 * total_files)
train_count = total_files - test_count - val_count  # Remaining 70%

# Split into groups
test_images = image_files[:test_count]
val_images = image_files[test_count : test_count + val_count]
train_images = image_files[test_count + val_count :]

# Function to move files while keeping images/labels in sync
def move_sync(image_paths, dest_image_dir, dest_label_dir):
    for img_path in image_paths:
        # Get the base filename (e.g., "2.png" → "2.txt")
        filename = os.path.basename(img_path)
        label_filename = os.path.splitext(filename)[0] + ".txt"
        
        # Determine current label location (train or val)
        current_label_dir = label_train if "train" in img_path else label_val
        label_path = os.path.join(current_label_dir, label_filename)
        
        # Move image
        shutil.move(img_path, os.path.join(dest_image_dir, filename))
        # Move corresponding label
        if os.path.exists(label_path):
            shutil.move(label_path, os.path.join(dest_label_dir, label_filename))
        else:
            print(f"Warning: Label {label_path} not found!")

# Move files to their new folders
move_sync(test_images, image_test, label_test)
move_sync(val_images, image_val, label_val)
move_sync(train_images, image_train, label_train)

print(f"Total files: {total_files}")
print(f"Train: {train_count} files ({train_count/total_files*100:.1f}%)")
print(f"Val: {val_count} files ({val_count/total_files*100:.1f}%)")
print(f"Test: {test_count} files ({test_count/total_files*100:.1f}%)")


Total files: 1437
Train: 1007 files (70.1%)
Val: 215 files (15.0%)
Test: 215 files (15.0%)


In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model (pre-trained on COCO)
model = YOLO('yolov8s.pt')  
path_to_data_yaml = "C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/Annotated_FruitNet/data.yaml"
# Train the model on the custom dataset
results = model.train(
    data=path_to_data_yaml,  
    epochs=50,                      # Number of epochs to train
    batch=16,                       # Batch size
    imgsz=256,                      # Image size
    plots=True,                     # Generate plots
    device=0                        # Use GPU (set -1 for CPU)
)

In [ ]:
# Load your trained model
model = YOLO('runs/detect/train2/weights/best.pt')  # Path to your best.pt
path_to_data_yaml = "C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/Annotated_FruitNet/data.yaml"

# Evaluate on the test set
results = model.val(
    data=path_to_data_yaml,
    split='test',  # Force evaluation on the test set (not validation)
    batch=16,
    imgsz=256,
    plots=True,
    verbose=True,
    device=0,       # GPU
    save_json=True  # Optional: Save results to JSON for analysis
)

# # Print key metrics
# print(f"mAP50-95: {results.box.map:.4f}")
# print(f"mAP50: {results.box.map50:.4f}")
# print(f"Precision: {results.box.p:.4f}")
# print(f"Recall: {results.box.r:.4f}")

## Data pipeline for vehicles

In [14]:
import yaml

data = {
    'path': "C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset",
    'test': 'images/test/',
    'train': 'images/train/',
    'val': 'images/test/',
    'nc': 6,
    'names': ['car', 'threewheel', 'bus', 'truck', 'motorbike', 'van']
}

with open('datasets/vehicle_dataset/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

In [13]:
import os
import random
import shutil

# Define paths
images_dir = "datasets/vehicle_dataset/images"
labels_dir = "datasets/vehicle_dataset/labels"

image_val = os.path.join(images_dir, "val")
image_test = os.path.join(images_dir, "test")

label_val = os.path.join(labels_dir, "val")
label_test = os.path.join(labels_dir, "test")

# Create test directories if they don't exist
os.makedirs(image_test, exist_ok=True)
os.makedirs(label_test, exist_ok=True)

# Get all image files from val (assuming .png/.jpg)
val_images = [os.path.join(image_val, f) for f in os.listdir(image_val) if f.endswith((".png", ".jpg", ".jpeg"))]
random.shuffle(val_images)  # Shuffle randomly

# Calculate 50% split
split_idx = len(val_images) // 2
test_images = val_images[:split_idx]
remaining_val_images = val_images[split_idx:]

# Move images + labels to test/val
for img_path in test_images:
    # Get corresponding label (e.g., 1.jpg -> 1.txt)
    filename = os.path.basename(img_path)
    label_name = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(label_val, label_name)

    # Move image to test
    shutil.move(img_path, os.path.join(image_test, filename))
    
    # Move label to test (if exists)
    if os.path.exists(label_path):
        shutil.move(label_path, os.path.join(label_test, label_name))
    else:
        print(f"Warning: Label {label_path} not found!")

print(f"Moved {len(test_images)} images (and labels) from val → test.")
print(f"Remaining val files: {len(remaining_val_images)} images.")

Moved 450 images (and labels) from val → test.
Remaining val files: 450 images.


## Training

In [20]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")
total_layers = sum(1 for _ in model.model.named_parameters())  # Count layers
print(f"Total trainable layers: {total_layers}")

Total trainable layers: 184


In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model (pre-trained on COCO)
model = YOLO('yolov8s.pt')  
path_to_data_yaml = "C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset/data.yaml"
# Train the model on the custom dataset
results = model.train(
    data=path_to_data_yaml,  
    epochs=100,                      # Number of epochs to train
    batch=100,                       # Batch size
    imgsz=640,                      # Image size
    freeze=30,                      # Freeze backbone
    plots=True,                     # Generate plots
    # resume=True,
    device=0                        # Use GPU (set -1 for CPU)
)

New https://pypi.org/project/ultralytics/8.3.102 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: task=detect, mode=train, model=runs/detect/cars_0_scratch/weights/best.pt, data=C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset/data.yaml, epochs=100, time=None, patience=100, batch=100, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=30, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augmen

RuntimeError: Dataset 'C://Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset/data.yaml' error  'C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset/data.yaml' does not exist

# Testing

In [17]:
# Load your trained model
model = YOLO('runs/detect/train/weights/best.pt')  # Path to your best.pt
path_to_data_yaml = "C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset/data.yaml"

# Evaluate on the test set
results = model.val(
    data=path_to_data_yaml,
    split='test',  # Force evaluation on the test set (not validation)
    batch=16,
    imgsz=256,
    plots=True,
    verbose=True,
    device=0,       # GPU
    save_json=True  # Optional: Save results to JSON for analysis
)

# # Print key metrics
# print(f"mAP50-95: {results.box.map:.4f}")
# print(f"mAP50: {results.box.map50:.4f}")
# print(f"Precision: {results.box.p:.4f}")
# print(f"Recall: {results.box.r:.4f}")

Ultralytics 8.3.99  Python-3.12.1 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\Sid\Documents\computer_vision\CNN.YOLO\datasets\vehicle_dataset\labels\test.cache... 450 images, 0 backgrounds, 0 corrupt: 100%|██████████| 450/450 [00:00<?, ?it/s]

val: WARNING  C:\Users\Sid\Documents\computer_vision\CNN.YOLO\datasets\vehicle_dataset\images\test\car55.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:03<00:00,  7.45it/s]


                   all        450        583      0.945        0.9      0.948      0.873
                   car        100        111      0.951      0.874      0.937      0.896
            threewheel         73        109      0.979      0.866      0.952       0.85
                   bus         81         90      0.986      0.944      0.978      0.945
                 truck         65         85      0.868      0.924      0.947      0.863
             motorbike         88        108      0.951      0.903      0.951      0.776
                   van         76         80      0.934      0.887      0.926      0.911
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 1.5ms postprocess per image
Saving runs\detect\val\predictions.json...
Results saved to runs\detect\val


In [21]:
# Load your trained model
model = YOLO('yolov8s.pt')  # Path to your best.pt
path_to_data_yaml = "C:/Users/Sid/Documents/computer_vision/CNN.YOLO/datasets/vehicle_dataset/data.yaml"

# Evaluate on the test set
results = model.val(
    data=path_to_data_yaml,
    split='test',  # Force evaluation on the test set (not validation)
    batch=16,
    imgsz=256,
    plots=True,
    verbose=True,
    device=0,       # GPU
    save_json=True  # Optional: Save results to JSON for analysis
)


Ultralytics 8.3.99  Python-3.12.1 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs


val: Scanning C:\Users\Sid\Documents\computer_vision\CNN.YOLO\datasets\vehicle_dataset\labels\test.cache... 450 images, 0 backgrounds, 0 corrupt: 100%|██████████| 450/450 [00:00<?, ?it/s]

val: WARNING  C:\Users\Sid\Documents\computer_vision\CNN.YOLO\datasets\vehicle_dataset\images\test\car55.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.80it/s]


                   all        450        583     0.0152     0.0725     0.0162     0.0155
                person        100        111          0          0   0.000201   8.31e-05
               bicycle         73        109          0          0    0.00597    0.00374
                   car         81         90    0.00329     0.0222    0.00219    0.00193
            motorcycle         65         85          0          0    0.00313    0.00276
              airplane         88        108          0          0          0          0
                   bus         76         80     0.0881      0.412     0.0859     0.0844
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Saving runs\detect\val2\predictions.json...
Results saved to runs\detect\val2
